# Just Another Ordinary Day In The Life Of A Quantum Software Engineer

Have you ever watched one of these reality shows about jobs and professions? Whether they show cops, firefighters, chefs, or even fishers, they all experience pretty exciting things every day.

Unsurprisingly, every kid wants to become one of them in the future. No kid dreams about sitting at the computer all day long drinking coffee. Ok, no kid except my two-year-old son. Whenever he sees me sitting at my desk, he insists on joining me, hitting the keys, and playing with the mouse.

Someone sitting at the computer all day long is not quite the material for a TV show. But maybe it is for a blog post?

A few weeks ago, I wrote about how you can find out whether you'd want to pursue a career in quantum computing. I claimed to work on a real problem to be the best way. So, I started to dissect IBM's Quantum Open Science Prize and to share my insights. First, we learned [what the challenge is actually about](https://towardsdatascience.com/a-first-glimpse-at-ibms-quantum-open-science-price-challenge-de4a2f41987e). It is about coping with an inherently unreliable real quantum computer. Then, we looked at [methods to reduce the effect of computation errors](https://pyqml.medium.com/how-not-to-use-todays-noisy-intermediate-scale-quantum-computers-a1c4d8551190). We focused on the [Clifford Data Regression (CDR)](https://pyqml.medium.com/mitigating-quantum-errors-using-clifford-data-regression-98ab663bf4c6) method that uses a machine learning model to predict and mitigate the noise. Finally, we used the open-source library Mitiq to effectively reduce errors in a [simulated environment](https://towardsdatascience.com/how-to-implement-quantum-error-mitigation-with-qiskit-and-mitiq-e2f6a933619c) and on a [real quantum computer](https://towardsdatascience.com/practical-error-mitigation-on-a-real-quantum-computer-41a99dddf740).

Of course, I did not invent any of these concepts or tools. Thus, I spent a significant amount of time reading sources. Surprisingly, most of these sources did not contain much math or physics. But the sources comprised mainly developer documentation and, in fact, Python source code. I worked myself through the CDR implementation in Mitiq and the state tomography implementation in Qiskit. 

Usually, I start with a conceptual overview of the topic. If you look at my [first post](https://pyqml.medium.com/mitigating-quantum-errors-using-clifford-data-regression-98ab663bf4c6) about CDR, you'll notice that it is very high-level. It doesn't contain a single mathematical formula. That's the level of understanding I feel comfortable with. I don't need math precisionto grasp what a concept is about. By contrast! I find math distracting when first trying to understand the essence of an idea.

So, with this basic understanding, I continued to recreate some existing code. In my post, [How To Implement Quantum Error Mitigation With Qiskit And Mitiq](https://towardsdatascience.com/how-to-implement-quantum-error-mitigation-with-qiskit-and-mitiq-e2f6a933619c), I followed the example provided by Mitiq. Yet, I did not copy and paste the source code. But, I adapted it to using Qiskit instead of Cirq.

Maybe it's only me. But trying to get some existing code to produce a particular behavior is the ordinary life of any software engineer. There's nothing quantumic in here. It doesn't matter if you try to dissect CDR or React (a Javascript library). Of course, someone more proficient in math and physics than me might choose another approach to solve the IBM challenge. But, I am a (quantum) **software engineer**. It determines how I try solving problems.

![](assets/yoda.png)

The way I try solving problems inevitably determines the struggles I'd need to cope with. It's like a self-fulfilling prophecy. I decided to follow a software engineering approach. So, I have to cope with software engineering problems.

The results of the previous post show an error reduction of more than 60%. Even though this is not as good as the mitigation of the simulated measurements we achieved in the [simulated environment](https://towardsdatascience.com/practical-error-mitigation-on-a-real-quantum-computer-41a99dddf740), it is a decent improvement over the unmitigated result. But what did we mitigate, essentially?

The CDR method reduces the deviations between the measured value of an observable and its actual value--the expectation value. In [this post](https://towardsdatascience.com/how-to-implement-quantum-error-mitigation-with-qiskit-and-mitiq-e2f6a933619c), we looked at observables in more detail. The expectation value of an observable is a single number. It is the probabilistic expected value of the measurement of an experiment. It is similar to a classical expectation value. For example, consider tossing a fair coin that lands on heads and tails equally likely. If you assign the value 1 to heads and 0 to tails, the expectation value is $0.5*1+0.5*0=0.5$.

For the expectation value describes the measurements, it does not describe the internal quantum state. And in quantum computing, the internal quantum state contains much more information than what we see through measures.

This is where the quantum state tomography comes into play. It is not possible to determine a quantum state in a single experiment. Let's think about the coin again. Let's say you would need to tell whether the coin lands on heads or tails while it is in the air. You may want to look at its rotational speed. If you know how many times it rotates while falling a certain distance, you could calculate the side up given the remaining distance. A quantum experiment tells us the rotational speed if you will. But it only tells us this speed around a specific axis.

But, what if the coin also rotates along its edge? This is another axis. To tell the rotational speed around the edge, we need to look at the coin from another perspective. We would need to look at it from another axis. This is what we do in quantum state tomography. To recreate the state of a single qubit, we need to look at it from three different angles. These are the X, Y, and Z axes. If we have two qubits, there are nine angles: XX, XY, XZ, YX, YY, YZ, ZX, ZY, and ZZ. We have to look at a two-qubit system from every single combination of single-qubit angles. The reason is entanglement. Entangled qubits build a quantum system that its separated components cannot describe entirely. But we need to look at it holistically.

If our quantum system consists of three qubits, there are 27 different combinations. With $n$ qubits, there are $3^n$ different perspectives.
Quantum state tomography works only for small quantum systems, such as the three-particle Heisenberg model Hamiltonian that IBM asks us to simulate.

Fortunately, Qiskit provides a function to perform a quantum state tomography. It consists of three steps.


In [ ]:
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# 1. Generate state tomography circuits to evaluate fidelity of simulation
st_qcs = state_tomography_circuits(qc, [qr[1], qr[3], qr[5]])

# ...

# 2. Fit state tomography results
tomo_fitter = StateTomographyFitter(result, st_qcs)
rho_fit = tomo_fitter.fit(method='# The expected final state;')

# 3. Compute fidelity
target_state = (One^One^Zero).to_matrix()  # The expected final state;
fid = state_fidelity(rho_fit, target_state)

First, we derive a set of 27 quantum circuits (`st_qcs`) from our primary circuit (`qc`). Each of these circuits "looks" at the primary circuit differently. Specifically, each of these circuits connects another observable to the primary circuit.

Second, we create a `StateTomographyFitter` object with Qiskit. We provide the measurement results from running the 27 circuits and an optimization method (here, least-square). Finally, we use Qiskit's `state_fidelity` function to calculate how close the measured quantum state is to the desired state (here, $|110>$)


So, how do we use CDR to mitigate the errors and improve the state fidelity assessed through quantum state tomography?

Conceptually, this is pretty straightforward. We replace the 27 state tomography circuits with 27 CDR instances. Each differs in the observable. Then, once we run the circuits, we can mitigate the measurement error using CDR. Finally, we rewrite the `StateTomographyFitter` class to work with the mitigated expectation values as the input.

That doesn't sound too complicated. However, rewriting this class is going to be an elaborate task.

I was speaking of software engineering problems.

Rewriting the `StateTomographyFitter` class would take a significant amount of time. Unfortunately, time is a very scarce resource. 
Moreover, I don't believe that IBM will accept a submission that builds upon rewriting their assessment tool. It could be hard for them to ensure that my rewritten `StateTomographyFitter` doesn't contain any cheats.

So, instead, I will refine my approach to work with the default `StateTomographyFitter`.
